# Decorator / Caching

In [35]:
import time
import datetime
import pickle
import os
import functools

In [36]:
cached_data = {}

def cache(ttl=15):
    """
    @ttl: time to live (seconds)
    """
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            global cached_data
            if os.path.exists("cache.pkl"):
                cached_data = pickle.load(open("cache.pkl", "rb"))
            
            now = datetime.datetime.now()
            key_name = f"{func.__name__}_{str(args)}_{str(kwargs)}"
            if key_name in cached_data.keys():
                if cached_data[key_name]["expiry"] >= now:
                    return cached_data[key_name]["value"]
                # else:
                #     del cached_data[key_name]

            value = func(*args, **kwargs)
            t_delta = datetime.timedelta(seconds=ttl)
            expiry = now + t_delta
            cached_data[key_name] = {}
            cached_data[key_name]["expiry"] = expiry
            cached_data[key_name]["value"] = value
            pickle.dump(cached_data, open("cache.pkl", "wb"))
            
            return value
        return wrapper
    return decorator

In [40]:
@cache(ttl=180)
def calculate(a, b):
    time.sleep(5)
    return a + b

def validate():
    time.sleep(2)
    return 5

In [41]:
func = validate
func.__name__

'validate'

In [42]:
func = calculate
func.__name__

'calculate'

In [5]:
%%time
calculate(1, 2)

CPU times: total: 0 ns
Wall time: 15.5 ms


3

In [81]:
%%time
calculate(2, 7)

CPU times: total: 0 ns
Wall time: 5.01 s


9

In [27]:
%%time
validate()

CPU times: total: 0 ns
Wall time: 0 ns


5

In [6]:
cached_data

{'calculate_(1, 2)_{}': {'expiry': datetime.datetime(2024, 7, 8, 10, 31, 40, 719532),
  'value': 3}}

In [34]:
args = (1, 2)
kwargs = {'a': 1, 'b': 2}
str(args), str(kwargs)

('(1, 2)', "{'a': 1, 'b': 2}")

In [54]:
now = datetime.datetime.now()
ttl = datetime.timedelta(seconds=20)
expiry = now + ttl
now, ttl, expiry

(datetime.datetime(2024, 7, 8, 10, 5, 44, 854611),
 datetime.timedelta(seconds=20),
 datetime.datetime(2024, 7, 8, 10, 6, 4, 854611))

In [ ]:
def retry(n=3):
    def decorator(func):
        def wrapper(*args, **kwargs):
            pass
            # logic comes here
        return wrapper
    return decorator

In [31]:
def f2(a, b):
    print(a)
    print(b)
    
def f(*args, **kwargs):
    print("Arguments:", args)
    print("KWArguments:", kwargs)
    # f2(*args, **kwargs)
    return None

# f(1, 5)
# f(a=1, b=5)
# f(1, b=5)

In [30]:
f(1, 2, 3, 4, 5, b=3, a=6, d='z')

Arguments: (1, 2, 3, 4, 5)
KWArguments: {'b': 3, 'a': 6, 'd': 'z'}
(1, 2, 3, 4, 5)
{'b': 3, 'a': 6, 'd': 'z'}


In [ ]:
f2((1, 2, 3, 4, 5),
   {'b': 3, 'a': 6, 'd': 'z'})